In [ ]:
# =============================================================
# Fast Colab Docking — SMILES input + pH(7.4) + Auto-Box + Prompts
# - apt-only: installs Vina + Open Babel (no Conda, no restart)
# - Ligand via SMILES (typed), 3D generation + pH 7.4 protonation
# - Receptor from uploaded PDB; auto-box from co-crystallized HET ligand
# - Prompts let you edit grid & Vina params (exhaustiveness, num_modes, etc.)
# - Outputs: out.pdbqt, vina_results.csv, vina.log
# - Created by Dr. Ahmed M.Sayed
# =============================================================

import os, csv, math, json, sys, re
from pathlib import Path
from typing import Optional, Tuple, Dict, List
from google.colab import files

# ---------- 0) Upload receptor ----------
print("📤 Upload your receptor PDB (.pdb). You may also upload a PDB with a co-crystallized ligand.")
uploaded = files.upload()
if not uploaded:
    raise SystemExit("No files uploaded.")

pdbs = [n for n in uploaded.keys() if n.lower().endswith(".pdb")]
if not pdbs:
    raise SystemExit("No .pdb file found in the upload.")
rec_pdb = pdbs[0]
print(f"✓ Receptor file: {rec_pdb}")

# ---------- 1) Install tools (apt binaries: fast & reliable on Colab) ----------
print("📦 Installing AutoDock Vina + Open Babel (Debian binaries)…")
os.system("apt-get -qq update")
if os.system("apt-get -qq install -y autodock-vina > /dev/null") != 0:
    raise SystemExit("Failed to install autodock-vina via apt.")
if os.system("apt-get -qq install -y openbabel > /dev/null") != 0:
    raise SystemExit("Failed to install openbabel via apt.")

# ---------- 2) Helpers ----------
def sh(cmd: str) -> int:
    print(f"$ {cmd}")
    return os.system(cmd)

def ask_float(prompt: str, default: float) -> float:
    try:
        s = input(f"{prompt} [{default}]: ").strip()
        return float(s) if s else float(default)
    except Exception:
        print("  -> Invalid input; keeping default.")
        return float(default)

def ask_int(prompt: str, default: int) -> int:
    try:
        s = input(f"{prompt} [{default}]: ").strip()
        return int(s) if s else int(default)
    except Exception:
        print("  -> Invalid input; keeping default.")
        return int(default)

# ---------- 3) Ligand via SMILES ----------
print("\n🔤 Enter ligand SMILES (e.g., CC(=O)Oc1ccccc1C(=O)O):")
smiles = input("SMILES: ").strip()
if not smiles:
    raise SystemExit("SMILES is required.")

# Optional pH (for both ligand and receptor protonation)
pH_default = 7.4
pH = ask_float("Protonation pH for preparation", pH_default)

# Write SMILES to a file to avoid shell-quoting issues
smiles_file = "ligand.smi"
with open(smiles_file, "w") as fh:
    fh.write(smiles + "\n")

lig_3d_sdf = "ligand_3d.sdf"
lig_pdbqt  = "ligand.pdbqt"

print("\n🧪 Generating 3D ligand and protonating at pH…")
# 3D + pH on ligand (SMILES -> SDF -> PDBQT)
# --gen3d builds coordinates; -p applies pH-based hydrogens (Open Babel)
sh(f'obabel -ismi "{smiles_file}" -osdf --gen3d -p {pH} -O "{lig_3d_sdf}"')
sh(f'obabel -isdf "{lig_3d_sdf}" -opdbqt -O "{lig_pdbqt}"')

# ---------- 4) Receptor protonation + PDBQT ----------
rec_pH74_pdb = "protein_pH.pdb"
rec_pdbqt    = "protein.pdbqt"

print("\n🧬 Preparing receptor: add H at pH and convert to rigid PDBQT (-xr)…")
sh(f'obabel -ipdb "{rec_pdb}" -opdb -p {pH} -O "{rec_pH74_pdb}"')
sh(f'obabel -ipdb "{rec_pH74_pdb}" -opdbqt -xr -O "{rec_pdbqt}"')

# ---------- 5) Auto-detect box from co-crystallized HET ligand ----------
ALLOW_WAT = {"HOH","WAT","OH2","H2O","DOD"}
ALLOW_IONS = {"ZN","MG","MN","FE","CA","NA","K","CL","CO","CU","NI","CD"}
def col(line: str, s: int, e: int) -> str:
    return line[s:e] if len(line) >= e else ""

def detect_ligand_bbox(pdb_path: str):
    groups: Dict[Tuple[str,str,str], List[Tuple[float,float,float]]] = {}
    with open(pdb_path, "r") as f:
        for raw in f:
            if not raw.startswith("HETATM"):
                continue
            resn = col(raw,17,20).strip()
            if resn in ALLOW_WAT or resn in ALLOW_IONS:
                continue
            chain = (col(raw,21,22).strip() or "-")
            resid = (col(raw,22,26).strip() or "?")
            try:
                x = float(col(raw,30,38).strip())
                y = float(col(raw,38,46).strip())
                z = float(col(raw,46,54).strip())
            except:
                continue
            key = (resn, chain, resid)
            groups.setdefault(key, []).append((x,y,z))
    if not groups:
        return None
    key = max(groups.keys(), key=lambda k: len(groups[k]))
    pts = groups[key]
    if len(pts) < 5:
        return None
    xs, ys, zs = zip(*pts)
    min_xyz = (min(xs), min(ys), min(zs))
    max_xyz = (max(xs), max(ys), max(zs))
    meta = {"resn": key[0], "chain": key[1], "resid": key[2], "natoms": len(pts)}
    return (min_xyz, max_xyz, meta)

auto = detect_ligand_bbox(rec_pdb)
if auto:
    (mnx,mny,mnz), (mxx,mxy,mxz), meta = auto
    cx = (mnx + mxx)/2.0; cy = (mny + mxy)/2.0; cz = (mnz + mxz)/2.0
    margin = 8.0
    sx = max(20.0, (mxx - mnx) + margin)
    sy = max(20.0, (mxy - mny) + margin)
    sz = max(20.0, (mxz - mnz) + margin)
    print(f"\n📍 Auto-box on HET ligand {meta['resn']} chain {meta['chain']} resid {meta['resid']} (atoms={meta['natoms']})")
else:
    print("\nℹ️ No co-crystallized HET ligand detected — using a generic cube (22 Å) centered at (0,0,0).")
    cx = cy = cz = 0.0
    sx = sy = sz = 22.0

# ---------- 6) Ask user to edit/confirm all docking parameters ----------
print("\n🛠️ Edit/confirm docking parameters (press Enter to accept the suggested value).")
cx = ask_float("center_x", cx)
cy = ask_float("center_y", cy)
cz = ask_float("center_z", cz)
sx = ask_float("size_x (Å)", sx)
sy = ask_float("size_y (Å)", sy)
sz = ask_float("size_z (Å)", sz)

EXHAUSTIVENESS = ask_int("exhaustiveness (search effort; time ↑)", 12)
NUM_MODES      = ask_int("num_modes (max. poses to output)", 10)
ENERGY_RANGE   = ask_float("energy_range (kcal/mol)", 3.0)
VERBOSITY      = ask_int("verbosity (0–4)", 1)

seed_in = input("seed (optional integer; blank=auto): ").strip()
cpu_in  = input("cpu  (optional threads; blank=auto): ").strip()
SEED = int(seed_in) if seed_in else None
CPU  = int(cpu_in)  if cpu_in  else None

# ---------- 7) Run Vina ----------
vina_out = "out.pdbqt"
vina_log = "vina.log"
cmd = (
    f'vina --receptor "{rec_pdbqt}" --ligand "{lig_pdbqt}" '
    f'--center_x {cx:.3f} --center_y {cy:.3f} --center_z {cz:.3f} '
    f'--size_x {sx:.3f} --size_y {sy:.3f} --size_z {sz:.3f} '
    f'--exhaustiveness {EXHAUSTIVENESS} --num_modes {NUM_MODES} '
    f'--energy_range {ENERGY_RANGE} --verbosity {VERBOSITY} --out "{vina_out}"'
)
if SEED is not None:
    cmd += f" --seed {SEED}"
if CPU is not None:
    cmd += f" --cpu {CPU}"

print("\n🚀 Running AutoDock Vina…")
print(cmd)
ret = os.system(cmd + f" | tee {vina_log}")
if ret != 0:
    print("❌ Vina returned a non-zero exit status. See vina.log.")

# ---------- 8) Parse results & offer downloads ----------
def parse_vina_results(pdbqt_file: str):
    rows = []; rank = 0
    if not Path(pdbqt_file).exists():
        return rows
    with open(pdbqt_file,"r") as f:
        for line in f:
            if line.startswith("REMARK VINA RESULT:"):
                parts = line.split()
                try:
                    energy  = float(parts[4])
                    rmsd_lb = float(parts[5])
                    rmsd_ub = float(parts[6])
                    rank += 1
                    rows.append((rank, energy, rmsd_lb, rmsd_ub))
                except:
                    pass
    return rows

vina_csv = "vina_results.csv"
rows = parse_vina_results(vina_out)
if rows:
    print("\nTop poses (rank, energy [kcal/mol], RMSD_lb, RMSD_ub):")
    for r in rows:
        print(f"{r[0]:2d}  {r[1]:8.3f}  {r[2]:6.3f}  {r[3]:6.3f}")
    with open(vina_csv, "w", newline="") as fh:
        w = csv.writer(fh)
        w.writerow(["rank","energy_kcal_per_mol","rmsd_lb","rmsd_ub"])
        w.writerows(rows)
else:
    print("(No REMARK VINA RESULT lines found in out.pdbqt)")

for f in [vina_out, vina_csv, vina_log]:
    if Path(f).exists():
        try:
            files.download(f)
        except Exception:
            pass

print("\n✅ Done.")
print(f"Center: ({cx:.3f}, {cy:.3f}, {cz:.3f})  Size: ({sx:.3f}, {sy:.3f}, {sz:.3f})  pH: {pH}")


📤 Upload your receptor PDB (.pdb). You may also upload a PDB with a co-crystallized ligand.


Saving protein.pdb to protein (1).pdb
✓ Receptor file: protein (1).pdb
📦 Installing AutoDock Vina + Open Babel (Debian binaries)…

🔤 Enter ligand SMILES (e.g., CC(=O)Oc1ccccc1C(=O)O):
SMILES: C1=CC(=C(C=C1C2=C(C(=O)C3=C(C=C(C=C3O2)O)O)O)O)O
Protonation pH for preparation [7.4]: 7.4

🧪 Generating 3D ligand and protonating at pH…
$ obabel -ismi "ligand.smi" -osdf --gen3d -p 7.4 -O "ligand_3d.sdf"
$ obabel -isdf "ligand_3d.sdf" -opdbqt -O "ligand.pdbqt"

🧬 Preparing receptor: add H at pH and convert to rigid PDBQT (-xr)…
$ obabel -ipdb "protein (1).pdb" -opdb -p 7.4 -O "protein_pH.pdb"
$ obabel -ipdb "protein_pH.pdb" -opdbqt -xr -O "protein.pdbqt"

ℹ️ No co-crystallized HET ligand detected — using a generic cube (22 Å) centered at (0,0,0).

🛠️ Edit/confirm docking parameters (press Enter to accept the suggested value).
center_x [0.0]: -9.924
center_y [0.0]: -1.466
center_z [0.0]: -2.947
size_x (Å) [22.0]: 25
size_y (Å) [22.0]: 25
size_z (Å) [22.0]: 25
exhaustiveness (search effort; time ↑

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Done.
Center: (-9.924, -1.466, -2.947)  Size: (25.000, 25.000, 25.000)  pH: 7.4
